In [ ]:
import numpy as np
import gymnasium as gym
from gymnasium.spaces import MultiBinary
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.logger import configure
import os
import pandas as pd
import matplotlib.pyplot as plt

# 스마트 팩토리 환경 생성
class SmartFactoryEnv(gym.Env):
    def __init__(self, n_machines=121, energy_limit=242):
        super(SmartFactoryEnv, self).__init__()
        self.n_machines = n_machines
        self.energy_limit = energy_limit

        # Multi-Binary 행동 공간
        self.action_space = MultiBinary(n_machines)

        # 관찰 공간
        self.observation_space = gym.spaces.Box(
            low=0, high=1, shape=(n_machines + 1,), dtype=float
        )

        self.state = None

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.state = np.zeros(self.n_machines + 1, dtype=float)
        return self.state, {}

    def step(self, action):
        energy_usage = np.dot(np.random.uniform(1, 2, self.n_machines), action)
        production = np.dot(np.random.uniform(1, 3, self.n_machines), action)
        penalty = max(0, energy_usage - self.energy_limit)
        reward = production - energy_usage - penalty

        self.state = np.concatenate([action, [energy_usage]])

        done = energy_usage > self.energy_limit * 2
        return self.state, reward, done, False, {}

    def render(self):
        print(f"Machines: {self.state[:-1]}, Energy Usage: {self.state[-1]}")

# 환경 설정
n_machines = 121  # MultiBinary 행동 공간 크기
env = DummyVecEnv([lambda: SmartFactoryEnv(n_machines=n_machines, energy_limit=242)])

# 로그 저장 디렉토리 설정 및 사용자 지정 이름 생성
log_dir = f"./logs_binary_{n_machines}/"
os.makedirs(log_dir, exist_ok=True)  # 디렉토리가 없으면 생성

# 사용자 정의 로거 설정
new_logger = configure(log_dir, ["stdout", "csv", "tensorboard"])

# PPO 모델 생성
model = PPO("MlpPolicy", env, verbose=1)
model.set_logger(new_logger)

# 학습
model.learn(total_timesteps=100000000)

# 로그 파일 경로 확인
print(f"Logs saved in directory: {log_dir}")



Logging to ./logs_binary_121/
Using cuda device


/usr/local/lib/python3.8/dist-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


-----------------------------
| time/              |      |
|    fps             | 2411 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 1652       |
|    iterations           | 2          |
|    time_elapsed         | 2          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.02327618 |
|    clip_fraction        | 0.283      |
|    clip_range           | 0.2        |
|    entropy_loss         | -83.9      |
|    explained_variance   | -6.35e-05  |
|    learning_rate        | 0.0003     |
|    loss                 | 1.16e+05   |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.0229    |
|    value_loss           | 2.46e+05   |
----------------------------------------
-----------------------------------------
| time/   

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# 로그 데이터 시각화
log_csv_path = os.path.join(log_dir, "progress.csv")  # log_dir은 이전에 정의된 디렉토리 변수

# CSV 파일 읽기 및 그래프 그리기
if os.path.exists(log_csv_path):
    data = pd.read_csv(log_csv_path)

    plt.figure(figsize=(10, 6))
    plt.plot(data["time/total_timesteps"], data["train/value_loss"], label="Value Loss")
    plt.plot(data["time/total_timesteps"], data["train/policy_gradient_loss"], label="Policy Gradient Loss")
    plt.xlabel("Timesteps")
    plt.ylabel("Loss")
    plt.title(f"Training Loss Over Timesteps (Binary {n_machines})")
    plt.legend()
    plt.grid()
    plt.show()
else:
    print("Log CSV file not found. Ensure training has completed successfully.")